In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import multiprocessing as mp
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import traceback
import sys

# Get the data from the reports table

In [2]:
# Install Webdriver
service = Service()

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://mrv.emsa.europa.eu/#public/emission-report")
time.sleep(60)

# html=driver.page_source
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="exportablegrid-1137-body"]')))


# Find and scrape the tables
tables = driver.find_element(By.XPATH, '//*[@id="exportablegrid-1137-body"]')
elements = tables.find_elements(By.TAG_NAME, 'table')

keys = ['Reporting Period', 'Version', 'Generation Date', 'File']
reports = []
for i, table in enumerate(elements):
    result_dict = {keys[i]: value for i, value in enumerate(table.text.split('\n')[1:])}
    reports.append(result_dict)
    
    print()
    
driver.quit()

In [3]:
reports

[{'Reporting Period': '2022',
  'Version': '106',
  'Generation Date': '14/11/2023',
  'File': '2022-v106-14112023-EU MRV Publication of information '},
 {'Reporting Period': '2021',
  'Version': '176',
  'Generation Date': '13/10/2023',
  'File': '2021-v176-13102023-EU MRV Publication of information '},
 {'Reporting Period': '2020',
  'Version': '194',
  'Generation Date': '11/10/2023',
  'File': '2020-v194-11102023-EU MRV Publication of information '},
 {'Reporting Period': '2019',
  'Version': '217',
  'Generation Date': '11/10/2023',
  'File': '2019-v217-11102023-EU MRV Publication of information '},
 {'Reporting Period': '2018',
  'Version': '270',
  'Generation Date': '11/10/2023',
  'File': '2018-v270-11102023-EU MRV Publication of information '}]

# Click on the link of the file to get the excel reports

In [ ]:
reports = [{'Reporting Period': '2022',
  'Version': '105',
  'Generation Date': '10/11/2023',
  'File': '2022-v105-10112023-EU MRV Publication of information'},
 {'Reporting Period': '2021',
  'Version': '176',
  'Generation Date': '13/10/2023',
  'File': '2021-v176-13102023-EU MRV Publication of information'},
 {'Reporting Period': '2020',
  'Version': '194',
  'Generation Date': '11/10/2023',
  'File': '2020-v194-11102023-EU MRV Publication of information'},
 {'Reporting Period': '2019',
  'Version': '217',
  'Generation Date': '11/10/2023',
  'File': '2019-v217-11102023-EU MRV Publication of information'},
 {'Reporting Period': '2018',
  'Version': '270',
  'Generation Date': '11/10/2023',
  'File': '2018-v270-11102023-EU MRV Publication of information'}]

In [ ]:
service = Service()

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://mrv.emsa.europa.eu/#public/emission-report")

# Adjust sleep time if needed, waiting for page elements to load
time.sleep(60)

for report in reports:
    print(f"Downloading the report {report['File']}")
    file = report['File']

    try:
        print('Waiting for the element to be clickable')
        # Use WebDriverWait to wait for the link to be clickable
        link = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.LINK_TEXT, file))
        )
        
        time.sleep(10)

        print('clicking the link')
        # Click on the link
        link.click()
        
        time.sleep(120)

    except Exception as e:
        print(f"An error occurred: {e}")
        print(traceback.format_exc())
    finally:
        driver.quit()